In [38]:
import folium
from ipywidgets import interact
import pandas as pd
import re
import requests
import random
import numpy as np

In [39]:
path2data = 'data/Alsace_166_176_307.csv'

path2data2 = 'data/data_alloffices.csv'

bat_data = pd.read_csv(path2data2, sep = ',', encoding='utf8')[:100]
# regex accents
#bat_data.Texte_Presentation = bat_data.Texte_Presentation.apply(lambda x: re.sub('\x92', " ", x))
num_col = ['Depenses_T2_2017', 'Depenses_T3_2017', 'Depenses_T5_2017', 'Depenses_HT2_Personnel_2017','Effectif_ETPT_2017', 'Actif_Immobilier', 'Actif_Vehicules']
#for col in num_col:
#    bat_data[col] = bat_data[col].apply(lambda x: np.float(re.sub(' ','', x)))

In [40]:
bat_data

,Unnamed: 0,Region,ID,Type,Fonction,Designation,Departement,Adesse,CodePostal,Ville,Pays,MinistereDIE,MinisterCode,Ministere,MinistereAll,Programmes,Latitude,Longitude
0,0,ALSACE,IB00100013234,Bureau,Immeuble de bureaux,REPRES.PERM.DE LA FRANCE AUPRES DU CONS,67,40 R DE VERDUN,67000.0,Strasbourg,France,Affaires trangres,16,Affaires trangres et dveloppement internati...,BureauImmeuble de bureauxAffaires trangres,156.0,48.588224,7.774069
1,1,ALSACE,IB00100016382,Btiment technique,Centre Informatique,E.S.I STRASBOURG-NEUDORF,67,23 R JOSEPH GUERBER,67000.0,Strasbourg,France,Budget - Comptes publics - Fonction publique,2,Economie et finances,Btiment techniqueCentre InformatiqueBudget - ...,105.0,48.564474,7.757484
2,2,ALSACE,IB00100204388,Bureau,Htel des finances,ANNEXE,67,10 R SIMONIS,67000.0,Strasbourg,France,Budget - Comptes publics - Fonction publique,2,Economie et finances,BureauHtel des financesBudget - Comptes publi...,156.0,48.589642,7.749546
3,3,ALSACE,IB00100017241,Bureau,Htel des impts,CENTRE DES FINANCES PUBLIQUES,67,35 AVENUE DES VOSGES,67000.0,Strasbourg,France,Budget - Comptes publics - Fonction publique,2,Economie et finances,BureauHtel des imptsBudget - Comptes publics...,156.0,48.589165,7.752262
4,4,ALSACE,IB00100017613,Bureau,Htel des impts,CFP STRASBOURG NEUDORF - BATIMENT A,67,10 R SIMONIS,67000.0,Strasbourg,France,Budget - Comptes publics - Fonction publique,2,Economie et finances,BureauHtel des imptsBudget - Comptes publics...,156.0,48.589642,7.749546
5,5,ALSACE,IB00100025735,Bureau,Immeuble de bureaux,DIRECTION DES DOUANES,67,11 AV DE LA LIBERTE,67000.0,Strasbourg,France,Budget - Comptes publics - Fonction publique,2,Economie et finances,BureauImmeuble de bureauxBudget - Comptes publ...,105.0,48.586420,7.757442
6,6,ALSACE,IB00100230483,Bureau,Immeuble de bureaux,ETABLISSEMENT DE SERVICES INFORMATIQUES GENERA...,67,17 RUE DU GENERAL PICQUART,67000.0,Strasbourg,France,Budget - Comptes publics - Fonction publique,2,Economie et finances,BureauImmeuble de bureauxBudget - Comptes publ...,105.0,48.581712,7.781570
7,7,ALSACE,IB00100036686,Bureau,Trsorerie,DIRECTION REGIONALE DES FINANCES PUBLIQUES D'A...,67,4 PL DE LA REPUBLIQUE,67000.0,Strasbourg,France,Budget - Comptes publics - Fonction publique,2,Economie et finances,BureauTrsorerieBudget - Comptes publics - Fon...,156.0,48.588144,7.753863
8,8,ALSACE,IB00100045307,Btiment technique,Garage,ANNEXE DU PALAIS DU RHIN/locaux DRAC,67,9 R DU GENERAL FRERE,67000.0,Strasbourg,France,Culture,12,Culture et communication,Btiment techniqueGarageCulture,105.0,48.588419,7.751965
9,9,ALSACE,IB00100038847,Bureau,Immeuble de bureaux,D.R.A.C ALSACE - S.D.A.P BAS-RHIN,67,2 PL DE LA REPUBLIQUE,67000.0,Strasbourg,France,Culture,12,Culture et communication,BureauImmeuble de bureauxCulture,105.0,48.587494,7.752793


In [41]:
textcase = {
    "judiciaire" : "Le programme budgétaire 166 « justice judiciaire » regroupe principalement les moyens des juridictions de l ordre judiciaire. Il compte la Cour de Cassation, 36 cours d appel et 836 juridictions du 1er degré (exemple tribunaux d instance).",
    "police nationale" : "Le programme budgétaire 176 « police nationale » regroupe principalement les moyens destinés à assurer la sécurité sur le territoire urbain. Il compte 661 commissariats.",
    "administration territoriale" : "Le programme budgétaire 307 « administration territoriale » regroupe les moyens destinés aux missions des préfectures et sous-préfectures soit  101 préfectures et 233 sous-préfectures."
}

## Explore dataframe

## Build map

In [42]:
map_styles = ['OpenStreetMap', 'CartoDB dark_matter', 'Mapbox Control Room', 'Stamen Watercolor']
folium_map = folium.Map(location=[48.5734, 7.7521113],
                        zoom_start=13,
                        tiles=map_styles[1],
                        width = '100%')

In [44]:
def latlon(df):
    col_ = list(df['Ville'].unique())
    res = pd.DataFrame(index=col_)
    for c in col_:
        lat = df.loc[df['Ville']==c, 'Latitude'].iloc[0]
        lon = df.loc[df['Ville']==c, 'Longitude'].iloc[0]
        res.loc[c, 'Latitude'] = lat
        res.loc[c, 'Longitude'] = lon
    return res

loc_cities = latlon(bat_data)
cities = list(loc_cities.index)


def list_min(df, colors_, names_):
    col_ = list(df["Ministere"].unique())
    res = pd.DataFrame(index=col_)
    i = 0
    for c in col_:
        res.loc[c, 'coul'] = colors_[i]
        res.loc[c, 'couleur'] = names_[i]
        i += 1
    return res

def list_prog(df):
    col_ = list(df["Libelle_Programme"].unique())
    col_.append('Tous')
    return col_

def list_fonction(df):
    col_ = list(df["Fonction"].unique())
    col_.append('Tous')
    return col_


colors = ["#FFCE00","#007849", "#E37222","#0375B4","#FFCE00","#0A8A9F"]
for element in bat_data['Ministere'].unique():
    r = lambda: random.randint(0,255)
    colors.append('#%02X%02X%02X' % (r(),r(),r()))

print(colors)

names = ['orange', 'green', 'tangerine', 'blue', 'yellow', 'teal', 'fbhejr', 'fvredb', 'dbfd', 'rfhjr', 'fvrdh', 'brfrbhd']
minis = list_min(bat_data, colors, names)
print(minis['couleur'])
fonction = list_fonction(bat_data)


# @interact(tiles=map_styles, cities_ = cities, fonction_ = fonction, zoom=(1, 18))
def create_map(tiles="Stamen Toner", zoom=13, cities_ = cities[0], fonction_= 'Tous'):
    
    try:
        lat = loc_cities.loc[cities_, 'Latitude']
        lon = loc_cities.loc[cities_, 'Longitude']
    except KeyError:
        lat = 48.5734
        lon = 7.7521113
        pass
    
    folium_map = folium.Map(location=[lat, lon], zoom_start=zoom, tiles=tiles, width = '100%')
    
    if fonction_ == 'Tous':
        for idx, row in bat_data.iterrows():

            # generate the popup message that is shown on click.
            #print(row)
            
            num_layus = "Pour répondre à ses missions, en 2017: \
            <ul> <li>Il a employé {etp} agents pour une dépense totale de {dep_perso}M€</li> \
            <li>Ses dépenses de fonctionnement ont représenté {dep_fonct}M€</li> \
            <li>Il a investi {invest}M€ en 2017</li> </ul>  \
            Il dispose aussi d’un patrimoine essentiellement composé d immeubles pour une valeur de {patrimoine}M€ \
            et véhicules pour une valeur de {vehicule}M€."
    
            popup_text = "<center><strong>{} </strong></center><br> <b>Ministere:</b> {}<br> \
            <b>Fonction:</b> {}"
    
            popup_text = popup_text.format(re.sub("'", ' ', row['Designation']).lower(),
                                  row["Ministere"],
                                  row["Fonction"])
                                  #row['Effectif_ETPT_2017'],
                                  # row['Texte_Presentation'],
                                  # num_layus.format(etp = int(row['Effectif_ETPT_2017']), dep_perso = int(row['Depenses_T2_2017']*1e-6), dep_fonct = int(row['Depenses_T3_2017']*1e-6), 
                                  #                  invest = int(row['Depenses_T5_2017']*1e-6), patrimoine = int(row['Actif_Immobilier']*1e-6), vehicule = int(row['Actif_Vehicules']*1e-6)))

            
            
            """
            popup_text = "<center><strong>{} </strong></center><br> <b>Ministere:</b> {}<br> <b>Fonction:</b> {}"
            popup_text = popup_text.format(re.sub("'", ' ', row['Designation']).lower(),
                                  row["Ministere"],
                                  row["Fonction"])
                                  # row['Effectif_ETPT_2017'],
                                  # row['Texte_Presentation'])

            
            """
            
            
            # radius of circles
            radius = 10
            color = minis.loc[row["Ministere"], 'coul']

            # add marker to the map
            folium.CircleMarker(location=(row["Latitude"],
                                          row["Longitude"]),
                                radius=radius,
                                color=color,
                                popup=popup_text,
                                fill=True).add_to(folium_map)
            
            
    else:
        bb = bat_data.loc[bat_data["Fonction"]==fonction_,]
        for idx, row in bb.iterrows():

            # generate the popup message that is shown on click.
            #print(row)
            num_layus = "Pour répondre à ses missions, en 2017: \
            <ul> <li>Il a employé {etp} agents pour une dépense totale de {dep_perso}M€</li> \
            <li>Ses dépenses de fonctionnement ont représenté {dep_fonct}M€</li> \
            <li>Il a investi {invest}M€ en 2017</li> </ul>  \
            Il dispose aussi d’un patrimoine essentiellement composé d immeubles pour une valeur de {patrimoine}M€ \
            et véhicules pour une valeur de {vehicule}M€."
            
            
            popup_text = "<center><strong>{} </strong></center><br> <b>Ministere:</b> {}<br> \
            <b>Fonction:</b> {}<br>"
    
            popup_text = popup_text.format(re.sub("'", ' ', row['Designation']).lower(),
                                  row["Ministere"],
                                  row["Fonction"])
                                  #row['Effectif_ETPT_2017'],
                                  # row['Texte_Presentation'],
                                  # num_layus.format(etp = int(row['Effectif_ETPT_2017']), dep_perso = int(row['Depenses_T2_2017']*1e-6), dep_fonct = int(row['Depenses_T3_2017']*1e-6), 
                                  #                  invest = int(row['Depenses_T5_2017']*1e-6), patrimoine = int(row['Actif_Immobilier']*1e-6), vehicule = int(row['Actif_Vehicules']*1e-6)))

            """
            
            popup_text = "<center><strong>{} </strong></center><br> <b>Ministere:</b> {}<br> <b>Fonction:</b> {}<br> <b>Effectifs 2017:</b> {} <br> {}"
            popup_text = popup_text.format(re.sub("'", ' ', row['Designation']).lower(),
                                  row["Ministere"],
                                  row["Fonction"],
                                  row['Effectif_ETPT_2017'],
                                  row['Texte_Presentation'])

            """
            
            # radius of circles
            radius = 10
            color = minis.loc[row["Ministere"], 'coul']

            # add marker to the map
            folium.CircleMarker(location=(row["Latitude"],
                                          row["Longitude"]),
                                radius=radius,
                                color=color,
                                popup=popup_text,
                                fill=True).add_to(folium_map)
            
    return folium_map

res = create_map(tiles="OpenStreetMap", zoom=13, cities_ = cities[0], fonction_= 'Tous')
res.save('mymap2.html')

['#FFCE00', '#007849', '#E37222', '#0375B4', '#FFCE00', '#0A8A9F', '#CDD2DB', '#1B283A', '#8DD550', '#42019C', '#6EAA76', '#47E162', '#11123F', '#2D157C']
Affaires trangres et dveloppement international             orange
Economie et finances                                            green
Culture et communication                                    tangerine
Environnement et mer                                             blue
Education nationale, enseignement suprieur et recherche       yellow
Intrieur                                                        teal
Justice                                                        fbhejr
Multi-occupation                                               fvredb
Name: couleur, dtype: object


In [45]:
folium_map.save('my_map.html')

In [19]:
bat_data

,ID,Type,Fonction,Designation,Latitude,Longitude,Adresse,CodePostal,Ville,Ministere,Programmes,Libelle_Programme,Texte_Presentation,Depenses_T2_2017,Depenses_T3_2017,Depenses_T5_2017,Depenses_HT2_Personnel_2017,Effectif_ETPT_2017,Actif_Immobilier,Actif_Vehicules
0,IB00100028657,Bureau,Tribunal,TRIBUNAL DE GRANDE INSTANCE DE STRASBOURG,48.587333,7.747495,QU FINKMATT,67000,Strasbourg,Justice,166,Justice judiciaire,Le programme budgétaire 166 « justice judiciai...,2.272061e+09,892981628.0,124801170.0,1.019850e+09,32488.0,3.445088e+09,2410917.0
1,IB00100036271,Bureau,Tribunal,TRIBUNAL D INSTANCE DE STRASBOURG,48.589145,7.746712,45 R DU FOSSE DES TREIZE,67000,Strasbourg,Justice,166,Justice judiciaire,Le programme budgétaire 166 « justice judiciai...,2.272061e+09,892981628.0,124801170.0,1.019850e+09,32488.0,3.445088e+09,2410917.0
2,IB00100028684,Bureau,Tribunal,CONSEIL DE PRUD'HOMMES DE STRASBOURG,48.589834,7.755428,19 AV DE LA PAIX,67000,Strasbourg,Justice,166,Justice judiciaire,Le programme budgétaire 166 « justice judiciai...,2.272061e+09,892981628.0,124801170.0,1.019850e+09,32488.0,3.445088e+09,2410917.0
3,IB00100035510,Bureau,Tribunal,TRIBUNAL ADMINISTRATIF STRASBOURG,48.591572,7.756567,31 AV DE LA PAIX,67000,Strasbourg,Services du premier ministre,166,Justice judiciaire,Le programme budgétaire 166 « justice judiciai...,2.272061e+09,892981628.0,124801170.0,1.019850e+09,32488.0,3.445088e+09,2410917.0
4,IB00100028712,Bureau,Tribunal,TRIBUNAL D INSTANCE DE MOLSHEIM,48.541025,7.494306,3 R DU MARECHAL KELLERMANN,67120,Molsheim,Justice,166,Justice judiciaire,Le programme budgétaire 166 « justice judiciai...,2.272061e+09,892981628.0,124801170.0,1.019850e+09,32488.0,3.445088e+09,2410917.0
5,IB00100029759,Bureau,Tribunal,TRIBUNAL D INSTANCE ET CONSEIL DE PRUD'HOMME D...,48.603212,7.754607,10 R DU TRIBUNAL,67300,Schiltigheim,Justice,166,Justice judiciaire,Le programme budgétaire 166 « justice judiciai...,2.272061e+09,892981628.0,124801170.0,1.019850e+09,32488.0,3.445088e+09,2410917.0
6,IB00100137195,Bureau,Tribunal,FUTUR SITE TI ET CONSEIL PRUDHOMME DE HAGUENAU,48.810646,7.790525,R DE LA REDOUTE,67500,Haguenau,Justice,166,Justice judiciaire,Le programme budgétaire 166 « justice judiciai...,2.272061e+09,892981628.0,124801170.0,1.019850e+09,32488.0,3.445088e+09,2410917.0
7,IB00100035570,Bureau,Tribunal,TRIBUNAL D INSTANCE DE SELESTAT,47.836293,7.175949,17 RUE DE LA PREMIERE ARMEE,67600,Selestat,Justice,166,Justice judiciaire,Le programme budgétaire 166 « justice judiciai...,2.272061e+09,892981628.0,124801170.0,1.019850e+09,32488.0,3.445088e+09,2410917.0
8,IB00100028628,Bureau,Tribunal,PALAIS DE JUSTICE DE SAVERNE,48.740552,7.365227,7 R DU TRIBUNAL,67700,Saverne,Justice,166,Justice judiciaire,Le programme budgétaire 166 « justice judiciai...,2.272061e+09,892981628.0,124801170.0,1.019850e+09,32488.0,3.445088e+09,2410917.0
9,IB00100035605,Bureau,Tribunal,TRIBUNAL D INSTANCE DE COLMAR,48.076186,7.357131,10 R DES AUGUSTINS,68000,Colmar,Justice,166,Justice judiciaire,Le programme budgétaire 166 « justice judiciai...,2.272061e+09,892981628.0,124801170.0,1.019850e+09,32488.0,3.445088e+09,2410917.0


In [150]:
"Ministere"
"Libelle_Programme"
"Fonction"

list_min = 


SyntaxError: invalid syntax (<ipython-input-150-04414fd85118>, line 5)